In [1]:
import os
import re
import pymupdf
import django_setup
from IPython.display import Markdown, display
from bs4 import BeautifulSoup as bs
from time import sleep
from pgvector.django import CosineDistance

from django.utils.datastructures import OrderedSet

from django.conf import settings
from django.contrib.contenttypes.models import ContentType

import google.generativeai as genai

gemini_api_key = settings.GEMINI_API_KEY

In [2]:
from cmj.search.models import Embedding
from sapl.compilacao.models import Dispositivo
from cmj.utils import clean_text

In [3]:
count_words = 0
chunks = []
for d in Dispositivo.objects.filter(
    ta_id=281,
    dispositivo_subsequente__isnull=True,
    tipo_dispositivo_id__gte=119
    ):
    if not d.nivel:
        continue
    relatives_chunks = d.render_relative_chunk(nivel=119)
    if relatives_chunks:
        ctext = clean_text(bs(relatives_chunks, 'html.parser').get_text())
        cw = len(ctext.split())
        chunks.append((d, cw, ctext))
        count_words += cw
    #print(d.id, bs(f'{d.rotulo} {d.texto}', 'html.parser').get_text()) #.prettify())

In [41]:
len(chunks), count_words

(983, 57641)

In [42]:
for c in chunks[0:1]:
    print(c)
    print('-' * 30)

(<Dispositivo: Art. 1º.>, 58, 'Art. 1º. - O Município de Jataí, divisão político-administrativa autônoma do Estado de Goiás e do qual é inseparável, reger-se-á por esta LEI ORGÂNICA e pelas leis que adotar, respeitadas as normas e princípios da Constituição da República e da Constituição do Estado.\nParágrafo Único - Todo o Poder emana do povo e em seu nome será exercido.')
------------------------------


In [4]:
genai.configure(api_key=gemini_api_key)

for d, cw, text in chunks:

    embeds = Embedding.objects.filter(
        object_id=d.id,
        content_type=ContentType.objects.get_for_model(d)
    )

    if embeds.exists():
        continue

    result = genai.embed_content(
            model='models/text-embedding-004',
            content=text)

    embed = result['embedding']

    embedding = Embedding()
    embedding.embedding = embed
    embedding.content_object = d

    embedding.metadata = {
        'model': 'models/text-embedding-004'
    }
    embedding.save()



In [5]:
Embedding.objects.count()

1774

In [6]:
def generate_query_embedding(query):
    result = genai.embed_content(
        model='models/text-embedding-004',
        content=query)

    return result['embedding']


def make_context(query_embedding):
    embeddings = Embedding.objects.annotate(
            distance=CosineDistance('embedding', query_embedding)
        ).order_by('distance')[:20]

    dispositivos = []
    for embed in embeddings:
        d = embed.content_object
        parents = d.get_parents_asc()
        parents.append(d)

        dispositivos.append(parents)

    dispositivos = sorted(dispositivos, key=lambda parents: (parents[-1].ta_id, parents[-1].ordem))

    dispositivos_filtered = []
    for i, parents in enumerate(dispositivos):
        if not i:
            ultimos_pks = set(map(lambda p: p.pk, parents))
            dispositivos_filtered.append(parents)
            continue

        atual_pks = set(map(lambda p: p.pk, parents))

        if ultimos_pks - atual_pks:
            dispositivos_filtered.append(parents)
            ultimos_pks = atual_pks


    context = ''

    for parents in dispositivos_filtered:
        parents.reverse()
        for p in parents:
            if p.tipo_dispositivo_id < 120:
                break
        disp_select = p
        text = disp_select.render_relative_chunk()
        context += f'{disp_select.ta}\n{text}\n\n'

    return context
    print(embed.distance)
    print(embed.content_object.dispositivo_pai.render_relative_chunk(nivel=119))


In [7]:
def get_model_configured():
    generation_config = {
      "temperature": 0.1,
      "top_p": 0.95,
      "top_k": 40,
      #"max_output_tokens": 8192,
      #"response_mime_type": "application/json",
      "response_mime_type": "text/plain",
    }

    model = genai.GenerativeModel(
      model_name="gemini-2.0-flash-exp",
      generation_config=generation_config,
    )

    return model

model = get_model_configured()


In [8]:
def make_prompt(query, context):
    context = clean_text(bs(context, 'html.parser').get_text())


    prompt = ("""Com base em <CONTEXTO></CONTEXTO> responda a pergunta de <PERGUNTA></PERGUNTA>

<CONTEXTO>{context}</CONTEXTO>

<PERGUNTA>{query}</PERGUNTA>

    """).format(query=query, context=context)

    while '  ' in prompt:
        prompt = prompt.replace('  ', ' ')

    return prompt

In [9]:
query = '''
quais conselhos devem ser criados no municipio?
'''

query_embedding = generate_query_embedding(query)
context = make_context(query_embedding)
prompt = make_prompt(query, context)
answer = model.generate_content(prompt)
display(Markdown(answer.text))

O texto fornecido não especifica quais conselhos devem ser criados no município. Ele detalha a organização e as competências da Câmara Municipal, suas sessões, comissões e processos legislativos, mas não aborda a criação de conselhos municipais.


In [10]:
print(prompt)

Com base em <CONTEXTO></CONTEXTO> responda a pergunta de <PERGUNTA></PERGUNTA>

<CONTEXTO>Lei Orgânica do Município nº 1 de 05 de Abril de 1990 -Título II - DA ORGANIZAÇÃO DOS PODERES Capítulo I - DO PODER LEGISLATIVO Seção I - DA CÂMARA MUNICIPAL Art. 13. - A Câmara Municipal reunir-se-á anualmente, em sessão legislativa, dividida em 2 (dois) períodos legislativos, compreendidos entre: 1º de fevereiro e 30 de junho, e 1º de agosto e 20 de dezembro.
§ 1º - As reuniões marcadas para essas datas serão transferidas para o primeiro dia útil subseqüente, quando caírem em sábados, domingos ou feriados.
§ 2º - A Câmara reunir-se-á em sessões ordinárias, extraordinárias ou solenes, conforme dispuser o Regimento Interno. O número de sessões ordinárias será, no mínimo, de cinco a cada mês.
§ 3º - A convocação extraordinária far-se-á:
a) - pelo Prefeito, quando entender necessário;
b) - pelo Presidente ou a requerimento da maioria dos seus membros, em caso de urgência ou interesse público relevan